# HP Tuning on Vertex AI
This notebook uses the Lending Club dataset to create an XGBoost model and run hyperparameter tuning in Vertex AI. 

# How to train your model on Vertex AI with HP tuning.
Using HP Tuning for training can be done in a few steps:
1. Create your python model file
    1. Add argument parsing for the hyperparameter values. (These values are chosen for you in this notebook)
    1. Add code to download your data from [Google Cloud Storage](https://cloud.google.com/storage) so that AI Platform can use it
    1. Add code to track the performance of your hyperparameter values.
    1. Add code to export and save the model to [Google Cloud Storage](https://cloud.google.com/storage) once AI Platform finishes training the model
1. Prepare a package
1. Submit the training job

# Prerequisites
Before you jump in, let’s cover some of the different tools you’ll be using to get HP tuning up and running on AI Platform. 

[Google Cloud Platform](https://cloud.google.com/) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training) provides a custom training service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.

[Overview of Hyperparameter Tuning](https://cloud.google.com/ml-engine/docs/tensorflow/hyperparameter-tuning-overview) - Hyperparameter tuning takes advantage of the processing infrastructure of Google Cloud Platform to test different hyperparameter configurations when training your model.

# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable AI Platform Training and Prediction and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [Install XGBoost](https://xgboost.readthedocs.io/en/latest/build.html) [Optional: used if running locally]
* [Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html) [Optional: used if running locally]
* [Install cloudml-hypertune](https://pypi.org/project/cloudml-hypertune/) [Optional: used if running locally]

In the cell below, **replace** the following highlighted elements:
* `project <PROJECT_ID>` - with this project id (i.e. ai-platform-demo)
* `bucket <BUCKET_ID>` - with your student id (i.e. maven-student01)
* `folder <FOLDER>` - with something about this exercise (i.e. lending_club)

In [1]:
# Replace <PROJECT_ID>, <BUCKET_ID>, and <FOLDER> with proper Project, Bucket ID, and Folder.
project = 'mwpmltr'
bucket = 'mwpmltr-vertex'
folder = 'vertex'
region = 'us-central1'

In [2]:
bucket_path=f'{bucket}/{folder}'
%env PROJECT_ID=$project
%env BUCKET_ID=$bucket
%env BUCKET_PATH=$bucket_path
!gsutil mb -c standard -l {region} gs://{bucket}

env: PROJECT_ID=mwpmltr
env: BUCKET_ID=mwpmltr-vertex
env: BUCKET_PATH=mwpmltr-vertex/vertex
Creating gs://mwpmltr-vertex/...
ServiceException: 409 A Cloud Storage bucket named 'mwpmltr-vertex' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


These variables will be needed for the following steps.
* `REGION <us-central1>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default 'us-central1'. The region is where the model will be deployed.
* `TRAINER_PACKAGE_PATH <./trainer>` - A packaged training application that will be staged in a Google Cloud Storage location. The model file created below is placed inside this package path.
* `MAIN_TRAINER_MODULE <trainer.task>` - Tells Vertex AI which file to execute. This is formatted as follows <folder_name.python_file_name>
* `HPTUNING_CONFIG <hptuning_config.yaml>` - Path to the job configuration file.
* `EXECUTOR_IMAGE_URL <us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest>` - The link to a pre-built XGBoost container image for training.

In [16]:
%env REGION us-central1
%env TRAINER_PACKAGE_PATH .
%env MAIN_TRAINER_MODULE trainer.task
%env HPTUNING_CONFIG hptuning_config.yaml
%env EXECUTOR_IMAGE_URI us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest

env: REGION=us-central1
env: TRAINER_PACKAGE_PATH=.
env: MAIN_TRAINER_MODULE=trainer.task
env: HPTUNING_CONFIG=hptuning_config.yaml
env: EXECUTOR_IMAGE_URI=us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest


## Download the data

 * File is `lending_club_data.tsv`

### Disclaimer
This dataset is provided by a third party. Google provides no representation,
warranty, or other guarantees about the validity or any other aspects of this dataset.

In [11]:
# Download the data
!gsutil cp gs://amazing-public-data/lending_club/lending_club_data.tsv gs://${BUCKET_PATH}/lending_club_data.tsv

Copying gs://amazing-public-data/lending_club/lending_club_data.tsv [Content-Type=text/tab-separated-values]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


# Part 1: Create your Python model file

We have created the Python model file (inside trainer folder) that we'll upload to Vertex AI. This is similar to your normal process for creating an XGBoost model. However, there are a few key differences:
1. Downloading the data from GCS at the start of your file, so that Vertex AI can access the data.
1. Exporting/saving the model to GCS at the end of your file, so that you can use it for predictions.
1. Define a command-line argument in your main training module for Vertex AI parameters and for each tuned hyperparameter.
1. Use the values passed in those arguments to set the corresponding Vertex AI parameters and hyperparameters in your application's XGBoost code.
1. Use `cloudml-hypertune` to track your training jobs metrics.

The code in this file first handles the parameters and hyperparameters passed to the file from Vertex AI. Then it loads the data into a pandas DataFrame that can be used by XGBoost. Then the model is fit against the training data and the metrics for that data are shared with Vertex AI. Lastly, Python's built in pickle library is used to save the model to a file that can be uploaded to [Vertex AI's prediction service](https://cloud.google.com/ml-engine/docs/scikit/getting-predictions#deploy_models_and_versions).

Note: In normal practice you would want to test your model locally on a small dataset to ensure that it works, before using it with your larger dataset on Vertex AI. This avoids wasted time and costs.

# Part 2: Create Trainer Package with Hyperparameter Tuning
The Trainer Package holds all your code and dependencies need to train your model on Vertex AI. 

The trainer folder and its contents can be referred to as the Trainer Package.

Next, we need to set the hp tuning values used to train our model. Check [HyperparameterSpec](https://cloud.google.com/ml-engine/reference/rest/v1/projects.jobs#HyperparameterSpec) for more info. 

In this config file several key things are set:
* `maxTrials` - How many training trials should be attempted to optimize the specified hyperparameters.
* `maxParallelTrials: 5` - The number of training trials to run concurrently. 
* `params` - The set of parameters to tune. These are the different parameters to pass into your model and the specified ranges you wish to try.
 * `parameterName` - The parameter name must be unique amongst all ParameterConfigs
 * `type` - The type of the parameter. [INTEGER, DOUBLE, ...]
 * `minValue` & `maxValue` - The range of values that this parameter could be. 
 * `scaleType` - How the parameter should be scaled to the hypercube. Leave unset for categorical parameters. Some kind of scaling is strongly recommended for real or integral parameters (e.g., UNIT_LINEAR_SCALE).
 
 
 
`hptuning_config.yaml` is the config file that we will be using here.

Lastly, we need to install the dependencies used in our model. Check [adding_standard_pypi_dependencies](https://cloud.google.com/ml-engine/docs/tensorflow/packaging-trainer#adding_standard_pypi_dependencies) for more info.

To do this, Vertex AI uses a setup.py file to install your dependencies.

# Part 3: Submit Training Job
Next we need to submit the job for training on Vertex AI. We'll use gcloud to submit the job which has the following flags:

* `display-name` - A name to use for the job (mixed-case letters, numbers, and underscores only, starting with a letter). In this case: `auto_mpg_hp_tuning_$(date +"%Y%m%d_%H%M%S")`
* `config` - Path to the job configuration file. This file should be a YAML document (JSON also accepted) containing a Job resource as defined in the API
* `region` - The Google Cloud Compute region where you want your job to run. You should run your training job in the same region as the Cloud Storage bucket that stores your training data. Select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'.
* `worker-pool-spec` - Define the worker pool configuration used by the custom job
    * `executor-image-uri` - A link to a [pre-built container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) to use for training.
    * `local-package-path` - The local path of a folder that contains training code.
    * `python-module` - The name of the main module in your trainer package. The main module is the Python file you call to start the application. Refer to Python Packages to figure out the module name.
    * `machine-type` - The type of the machine.
* `args` - A comma separated list of custom parameters used in the Python file

Note: Check to make sure gcloud is set to the current PROJECT_ID

In [12]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Submit the training job.

In [17]:
import time
from datetime import datetime, timedelta

In [24]:
# Workaround for Docker in Vertex AI Notebook
!sudo chmod 666 /var/run/docker.sock

In [25]:
now=(datetime.now() + timedelta(hours=-5)).strftime("%Y%m%d_%H%M%S") # Central Time
%env JOB_NAME=lending_club_job_{now}

# This step will fail, but will create a container for training
!gcloud beta ai custom-jobs create \
  --display-name $JOB_NAME \
  --config $HPTUNING_CONFIG \
  --region $REGION \
  --worker-pool-spec=executor-image-uri=$EXECUTOR_IMAGE_URI,local-package-path=$TRAINER_PACKAGE_PATH,python-module=$MAIN_TRAINER_MODULE,machine-type=n1-standard-4 \
  --args=project-id=$PROJECT_ID,bucket-name=$BUCKET_PATH

env: JOB_NAME=lending_club_job_20210724_231846
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Sending build context to Docker daemon  68.66kB
Step 1/10 : FROM us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest
 ---> 6da3b9be283f
Step 2/10 : RUN mkdir -m 777 -p /usr/app /home
 ---> Running in a12f0419fb58
Removing intermediate container a12f0419fb58
 ---> 466e5ffe6846
Step 3/10 : WORKDIR /usr/app
 ---> Running in 069e4134d8e7
Removing intermediate container 069e4134d8e7
 ---> 28ba091bb396
Step 4/10 : ENV HOME=/home
 ---> Running in edc354b2ee53
Removing intermediate container edc354b2ee53
 ---> d61e64332e8f
Step 5/10 : ENV PYTHONDONTWRITEBYTECODE=1
 ---> Running in 27a4b47111b8
Removing intermediate container 27a4b47111b8
 ---> cf43875da1bb
Step 6/10 : RUN rm -rf /var/sitecustomize
 ---> Running in 00c9f248d3d6
Removing intermediate container 00c9f248d3d6
 ---> 9754982f5ebf
Step 7/10 : COPY ["./setup.py", "./setup.py"]
 ---> 085874624eea
Step 8/10 : RUN pip ins

In [27]:
!gcloud beta ai hp-tuning-jobs create \
  --display-name $JOB_NAME \
  --config $HPTUNING_CONFIG \
  --region $REGION

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Hyperparameter tuning job [7361709735030030336] submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud beta ai hp-tuning-jobs describe 7361709735030030336

Job State: JOB_STATE_PENDING


In [30]:
%env JOB_NAME=7361709735030030336
# Model should exit with status "SUCCEEDED"
# !gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"
cmd = 'gcloud beta ai hp-tuning-jobs describe $JOB_NAME --region $REGION --format="value(state)"'
for i in range(20):
    time.sleep(10)
    !{cmd}

env: JOB_NAME=7361709735030030336
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
JOB_STATE_RUNNING
Using endpoint [https://us-central1-aiplatform.googleapis.com/]


# [Optional] Part 4: StackDriver Logging
You can view the logs for your training job:
1. Go to https://console.cloud.google.com/
1. Select "Logging" in left-hand pane
1. In left-hand pane, go to "AI Platform" and select Jobs
1. In filter by prefix, use the value of $JOB_NAME to view the logs

On the logging page of your model, you can view the different results for each HP tuning job. 

Example:
```
{
  "trialId": "15",
  "hyperparameters": {
    "booster": "dart",
    "max_depth": "7",
    "n_estimators": "102"
  },
  "finalMetric": {
    "trainingStep": "1000",
    "objectiveValue": 0.9259230441279733
  }
}
```

# [Optional] Part 5: Verify Model File in GCS
View the contents of the destination model folder to verify that all 5 model files have indeed been uploaded to GCS.

Note: The model can take a few minutes to train and show up in GCS.

An example of the log output at the end of hyperparameter training showing the final AUC score and parameters chosen to achieve it:
